In [ ]:
%pip install gensim transformers nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 857.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 16.0 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.14.1
    Uninstalling scipy-1.14.1:
      Successfully uninstalled scipy-1.14.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.


In [ ]:
import gensim.downloader as api
from transformers import pipeline
import nltk
import string
from nltk.tokenize import word_tokenize
nltk.download('punkt_tab')

print("--> Loading pre-trained word vectors")
word_vectors = api.load("glove-wiki-gigaword-100")
print("--> Loaded pre-trained word vectors successfully!")

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


--> Loading pre-trained word vectors
[==================================================] 100.0% 128.1/128.1MB downloaded
--> Loaded pre-trained word vectors successfully!


In [ ]:
def replace_keyword_in_prompt(prompt, keyword, word_vectors, topn=1):
  words = word_tokenize(prompt)
  enriched_words = []
  for word in words:
    cleaned_word = word.lower().strip(string.punctuation)
    if cleaned_word == keyword.lower():
      try:
        similar_words = word_vectors.most_similar(cleaned_word, topn=topn)
        if similar_words:
          replacement_word = similar_words[0][0]
          print(f"Replacing '{word}' -> '{replacement_word}'")
          enriched_words.append(replacement_word)
        else:
          enriched_words.append(word)
      except KeyError:
        print(f"'{cleaned_word}' not found in vocabulary.")
        enriched_words.append(word)
    else:
      enriched_words.append(word)
  enriched_prompt = " ".join(enriched_words)
  print(f"Enriched prompt: {enriched_prompt}")
  return enriched_prompt

In [ ]:
print("Loading GPT-2 model")
generator = pipeline("text-generation", model="gpt2")
print("Loaded GPT-2 model successfully!")

Loading GPT-2 model


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu


Loaded GPT-2 model successfully!


In [ ]:
def generate_response(prompt, max_length=100):
  try:
    response = generator(prompt, max_length=max_length, num_return_sequences=1)
    return response[0]['generated_text']
  except Exception as e:
    print(f"Error generating response: {e}")
    return None


original_prompt = "who is queen?"
print(f"\n\nOriginal prompt: {original_prompt}")
key_term = "queen"
enriched_prompt = replace_keyword_in_prompt(original_prompt, key_term, word_vectors)

print("\n\n ---- ORIGINAL PROMPT ---- ")
print(f"\n PROMPT: \n{original_prompt}")


original_response = generate_response(original_prompt)
print(f"\n RESPONSE: \n{original_response}")

print("\n\n ---- ENRICHED PROMPT ----")
print(f"\n PROMPT: {enriched_prompt}")
enriched_response = generate_response(enriched_prompt)
print(f"\n RESPONSE: \n{enriched_response}")


print("\n\n ---- COMPARISON ---- ")
print("\nOriginal prompt response length: ", len(original_response))
print("Enriched prompt response length: ", len(enriched_response))
print("\nOriginal prompt response detail: ", original_response.count("."))
print("Enriched prompt response detail: ", enriched_response.count("."))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.




Original prompt: who is queen?
Replacing 'queen' -> 'princess'
Enriched prompt: who is princess ?


 ---- ORIGINAL PROMPT ---- 

 PROMPT: 
who is queen?


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



 RESPONSE: 
who is queen? Oh that's what a lot of people say.)

I am on the verge of going "yes" to Trump; I expect that I shall end up with a man or woman who embodies this particular worldview and beliefs. But, as in my book, I am too busy to write in a way that would go as far as the left or the "right." I know that if I make a living with it and do what a fair and balanced press would advise, I


 ---- ENRICHED PROMPT ----

 PROMPT: who is princess ?

 RESPONSE: 
who is princess ? )? ( princess = ( "Pony Spice") ) || princess is not a good person;

( { princess = ( "Bunny Spice" [1])? ( bunny = ( "Wife" [2])? (( "Bloke" )[6] ) ) || ( ( "The Duchess" ) [4] ) || ( "Queen" ) [6] ) ) ) ;

// This pattern is pretty easy to follow. Now


 ---- COMPARISON ---- 

Original prompt response length:  389
Enriched prompt response length:  271

Original prompt response detail:  3
Enriched prompt response detail:  1


VIVA

----
1. What is the significance of Cohere, LangChain, and Pydantic?

- Cohere: Provides NLP APIs including embeddings, classification, and summarization via LLMs. Used for model inference.

- LangChain: Framework to build applications powered by LLMs. It connects LLMs with tools like memory, APIs, and agents.

- Pydantic: Used for data validation and parsing using Python type hints. Useful in defining schemas and ensuring structured output from LLMs.

----

2. Explain the techniques used in word embeddings.

- Traditional: One-hot encoding, Bag of Words (BoW), TF-IDF.

- Neural: Word2Vec (CBOW, Skip-gram), GloVe (co-occurrence matrix), FastText (subword n-grams), BERT (contextual embeddings).

----

3. Why do we use word embeddings?/ Need for word embeddings.

- To reduce dimensionality and sparsity.

- To capture semantic and syntactic meanings.

- To enable words with similar meanings to have similar representations.

----

4. Discuss the real-world applications of LLMs and their limitations.

- Applications: Chatbots, summarization, code generation, sentiment analysis, content creation.

- Limitations: Bias in training data, hallucination, high computational cost, lack of real-time awareness.

----

5. Which model is used for summarization?

- Pre-trained summarization model from Hugging Face (facebook/bart-large-cnn)

----

6. Explain the BART model in detail.

- BART (Bidirectional and Auto-Regressive Transformer) combines BERT (encoding) and GPT (decoding).

- It is a sequence-to-sequence model used for text generation, summarization, translation, etc.

- Trained by corrupting text and learning to reconstruct it.

----

7. What is sentiment analysis and its applications?

- It is the process of identifying sentiment (positive, negative, neutral) from text.

- Applications: Customer feedback, brand monitoring, political analysis, market research.

----

8. Discuss and explain the significance of the parameter perplexity in t-SNE.

- hyperparameter that defines the effective number of neighbors.

- Controls the balance between local(less perplexity) vs. global(more perplexity) structure.

- Should be less than the number of data points; typical range: 5–50.

----

9. Describe the algorithm (step-by-step, in words) for building an IPC chatbot.

a. Download the Indian Penal Code document.

b. Preprocess and split the document into retrievable chunks.

c. Use embeddings to store the chunks in a vector store.

d. Accept user queries.

e. Retrieve relevant sections using similarity search.

f. Use LLM (via LangChain) to answer based on the retrieved context.

----

10. Discuss PCA and t-SNE.

- PCA: Linear, preserves global variance, faster, used for large datasets.

- t-SNE: Non-linear, preserves local relationships, ideal for visualizing word clusters in small data.

- Used to visualize high-dimensional word embeddings in 2D/3D.

----

11. What are the uses of prompt engineering?

- To control LLM outputs by carefully designing the input prompts.

- Used in chatbots, summarization, translation, data extraction, and few-shot learning.

----